In [1]:
import mne
import numpy as np
import os
from glob import glob
from tqdm import tqdm


from utils import read_TUH_edf


# TUEV

In [2]:
# TUEV_dir_path = "/nobackup/tsal-tmp/tuh_eeg_seizure/v2.0.0/edf/train/"
TUEV_dir_path = "/scratch/s194101/TUEV_processed"
save_dir = "/scratch/s194101/concepts/TUEV/"

edf_files = glob(f"{TUEV_dir_path}/*/*.edf")
print(f"{len(edf_files)=}")
print(edf_files[:10])

len(edf_files)=357
['/scratch/s194101/TUEV_processed/00000081/00000081_00000005.edf', '/scratch/s194101/TUEV_processed/00000878/00000878_00000001.edf', '/scratch/s194101/TUEV_processed/00002316/00002316_00000001.edf', '/scratch/s194101/TUEV_processed/00000422/00000422_00000001.edf', '/scratch/s194101/TUEV_processed/00003034/00003034_00000001.edf', '/scratch/s194101/TUEV_processed/00003693/00003693_00000001.edf', '/scratch/s194101/TUEV_processed/00000388/00000388_00000001.edf', '/scratch/s194101/TUEV_processed/00003247/00003247_00000001.edf', '/scratch/s194101/TUEV_processed/00001824/00001824_00000001.edf', '/scratch/s194101/TUEV_processed/00002875/00002875_00000007.edf']


In [3]:
sample_file = edf_files[0]
sample_file = '/scratch/s194101/TUEV_processed/00000878/00000878_00000001.edf'



In [4]:
raw = mne.io.read_raw_edf(sample_file, preload=True, verbose=False)
raw.ch_names

['EEG FP1-REF',
 'EEG FP2-REF',
 'EEG F3-REF',
 'EEG F4-REF',
 'EEG C3-REF',
 'EEG C4-REF',
 'EEG P3-REF',
 'EEG P4-REF',
 'EEG O1-REF',
 'EEG O2-REF',
 'EEG F7-REF',
 'EEG F8-REF',
 'EEG T3-REF',
 'EEG T4-REF',
 'EEG T5-REF',
 'EEG T6-REF',
 'EEG A1-REF',
 'EEG A2-REF',
 'EEG FZ-REF',
 'EEG CZ-REF',
 'EEG PZ-REF',
 'EEG ROC-REF',
 'EEG LOC-REF',
 'EEG EKG1-REF',
 'EEG T1-REF',
 'EEG T2-REF',
 'PHOTIC-REF']

In [19]:
raw.annotations

<Annotations | 292 segments: artf (1), bckg (283), eyem (8)>

In [5]:
channel_map = {
        'EEG C3-REF': 'C3', 'EEG P4-REF': 'P4', 'EEG T5-REF': 'T5', 'EEG F8-REF': 'F8', 'EEG F7-REF': 'F7',
        'EEG C4-REF': 'C4', 'EEG PZ-REF': 'Pz', 'EEG FP2-REF': 'Fp2', 'EEG F4-REF': 'F4', 'EEG F3-REF': 'F3',
        'EEG T6-REF': 'T6', 'EEG CZ-REF': 'Cz', 'EEG O2-REF': 'O2', 'EEG O1-REF': 'O1', 'EEG T2-REF': 'F10',
        'EEG T1-REF': 'F9', 'EEG T4-REF': 'T4', 'EEG P3-REF': 'P3', 'EEG FZ-REF': 'Fz', 'EEG T3-REF': 'T3',
        'EEG FP1-REF': 'Fp1', 'EEG C4-LE': 'C4', 'EEG P3-LE': 'P3', 'EEG FZ-LE': 'Fz', 'EEG F3-LE': 'F3',
        'EEG FP1-LE': 'Fp1', 'EEG T6-LE': 'T6', 'EEG CZ-LE': 'Cz', 'EEG F8-LE': 'F8', 'EEG O1-LE': 'O1',
        'EEG PZ-LE': 'Pz', 'EEG C3-LE': 'C3', 'EEG FP2-LE': 'Fp2', 'EEG O2-LE': 'O2', 'EEG F7-LE': 'F7',
        'EEG T1-LE': 'T9', 'EEG T2-LE': 'F10', 'EEG P4-LE': 'P4', 
    }

channel_map_sub = {k: v for k, v in channel_map.items() if k in raw.ch_names}

In [6]:
channel_map_sub

{'EEG C3-REF': 'C3',
 'EEG P4-REF': 'P4',
 'EEG T5-REF': 'T5',
 'EEG F8-REF': 'F8',
 'EEG F7-REF': 'F7',
 'EEG C4-REF': 'C4',
 'EEG PZ-REF': 'Pz',
 'EEG FP2-REF': 'Fp2',
 'EEG F4-REF': 'F4',
 'EEG F3-REF': 'F3',
 'EEG T6-REF': 'T6',
 'EEG CZ-REF': 'Cz',
 'EEG O2-REF': 'O2',
 'EEG O1-REF': 'O1',
 'EEG T2-REF': 'F10',
 'EEG T1-REF': 'F9',
 'EEG T4-REF': 'T4',
 'EEG P3-REF': 'P3',
 'EEG FZ-REF': 'Fz',
 'EEG T3-REF': 'T3',
 'EEG FP1-REF': 'Fp1'}

In [7]:
mne.datasets.eegbci.standardize(raw)

raw = raw.rename_channels(channel_map_sub)

In [8]:
montage = mne.channels.make_standard_montage('standard_1020')
# set the montage
raw = raw.set_montage(montage, on_missing='ignore', verbose=False)

In [9]:
raw = raw.pick(list(channel_map_sub.values()))
raw.ch_names

['C3',
 'P4',
 'T5',
 'F8',
 'F7',
 'C4',
 'Pz',
 'Fp2',
 'F4',
 'F3',
 'T6',
 'Cz',
 'O2',
 'O1',
 'F10',
 'F9',
 'T4',
 'P3',
 'Fz',
 'T3',
 'Fp1']

In [10]:
raw = raw.set_eeg_reference(ref_channels='average', projection=True)
raw.apply_proj()

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


<RawEDF | 00000878_00000001.edf, 21 x 301000 (1204.0 s), ~48.3 MB, data loaded>

In [11]:
raw = raw.resample(256)

In [12]:
def read_tuev_edf(file_path, process=True, notch_filter=False):
    raw = mne.io.read_raw_edf(file_path, preload=True, verbose=False)

    if process:
        channel_map = {
            'EEG C3-REF': 'C3', 'EEG P4-REF': 'P4', 'EEG T5-REF': 'T5', 'EEG F8-REF': 'F8', 'EEG F7-REF': 'F7',
            'EEG C4-REF': 'C4', 'EEG PZ-REF': 'Pz', 'EEG FP2-REF': 'Fp2', 'EEG F4-REF': 'F4', 'EEG F3-REF': 'F3',
            'EEG T6-REF': 'T6', 'EEG CZ-REF': 'Cz', 'EEG O2-REF': 'O2', 'EEG O1-REF': 'O1', 'EEG T2-REF': 'F10',
            'EEG T1-REF': 'F9', 'EEG T4-REF': 'T4', 'EEG P3-REF': 'P3', 'EEG FZ-REF': 'Fz', 'EEG T3-REF': 'T3',
            'EEG FP1-REF': 'Fp1', 'EEG C4-LE': 'C4', 'EEG P3-LE': 'P3', 'EEG FZ-LE': 'Fz', 'EEG F3-LE': 'F3',
            'EEG FP1-LE': 'Fp1', 'EEG T6-LE': 'T6', 'EEG CZ-LE': 'Cz', 'EEG F8-LE': 'F8', 'EEG O1-LE': 'O1',
            'EEG PZ-LE': 'Pz', 'EEG C3-LE': 'C3', 'EEG FP2-LE': 'Fp2', 'EEG O2-LE': 'O2', 'EEG F7-LE': 'F7',
            'EEG T1-LE': 'T9', 'EEG T2-LE': 'F10', 'EEG P4-LE': 'P4', 
        }

        channel_map_sub = {k: v for k, v in channel_map.items() if k in raw.ch_names}
        
        # standardize the raw data
        mne.datasets.eegbci.standardize(raw)

        # rename the channels that are in the channel_map_sub
        raw = raw.rename_channels(channel_map_sub)


        # make standard 10-20 montage
        montage = mne.channels.make_standard_montage('standard_1020')
        # set the montage
        raw = raw.set_montage(montage, on_missing='ignore', verbose=False)
        
        # this is where we actually make it so we are only left with the channels we want
        raw = raw.pick(list(channel_map_sub.values()))

        # Set the average reference for the raw data
        raw = raw.set_eeg_reference(ref_channels='average', projection=True, verbose=False)

        # where we actually apply the projection from above
        raw.apply_proj(verbose=False)

        # resample the data to 256 Hz
        raw = raw.resample(256)

        if notch_filter:
            raw = raw.notch_filter(60.0, fir_design='firwin', verbose=False)


    return raw

In [13]:
raw_processed = read_tuev_edf(sample_file, process=True, notch_filter=False)
raw_processed

<RawEDF | 00000878_00000001.edf, 21 x 308224 (1204.0 s), ~49.4 MB, data loaded>

In [18]:
raw_processed.annotations

<Annotations | 292 segments: artf (1), bckg (283), eyem (8)>

## make each annotations its own pickle file

In [15]:
raw_processed.annotations[3]

OrderedDict([('onset', 1.5),
             ('duration', 1.0),
             ('description', 'bckg'),
             ('orig_time',
              datetime.datetime(2012, 2, 15, 14, 8, 7, tzinfo=datetime.timezone.utc))])

In [16]:
raw_processed

<RawEDF | 00000878_00000001.edf, 21 x 308224 (1204.0 s), ~49.4 MB, data loaded>

In [17]:
raw_processed.annotations

<Annotations | 292 segments: artf (1), bckg (283), eyem (8)>

In [49]:
def get_window(raw, annotation, window_size):
    """
    given a raw object and an annotation, return a window of the data
    """

    onset, duration, description, _ = annotation.values()

    start = int(onset * raw.info['sfreq'])
    
    # end is the start plus the window size
    end = start + int(window_size * raw.info['sfreq'])

    # if the end is longer than the data we can not procede, otherwise we will get an error
    if end > raw.get_data().shape[1]:
        return None

    window = raw.copy().get_data()[:, start:end]

    return window



save_dir = "/scratch/s194101/concepts/TUEV"


for idx, annotation in enumerate(raw_processed.annotations):
    onset, duration, description, _ = annotation.values()
    print(f"{onset=}, {duration=}, {description=}")

    # separate the file into windows that are 
    window = get_window(raw_processed, annotation, 60)

    # save the window as a pickle file, in a folder based on its label

    save_path = f"{save_dir}/{description}/{os.path.basename(sample_file).split('.')[0]}_{idx}.pkl"
    print(save_path)
    



onset=1.4, duration=1.0, description='eyem'
/scratch/s194101/concepts/TUEV/eyem/00000878_00000001_0.pkl
onset=1.5, duration=1.0, description='artf'
/scratch/s194101/concepts/TUEV/artf/00000878_00000001_1.pkl
onset=1.5, duration=1.0, description='bckg'
/scratch/s194101/concepts/TUEV/bckg/00000878_00000001_2.pkl
onset=1.5, duration=1.0, description='bckg'
/scratch/s194101/concepts/TUEV/bckg/00000878_00000001_3.pkl
onset=1.5, duration=1.0, description='eyem'
/scratch/s194101/concepts/TUEV/eyem/00000878_00000001_4.pkl
onset=1.5, duration=1.0, description='eyem'
/scratch/s194101/concepts/TUEV/eyem/00000878_00000001_5.pkl
onset=1.5, duration=1.0, description='eyem'
/scratch/s194101/concepts/TUEV/eyem/00000878_00000001_6.pkl
onset=2.9, duration=1.0, description='eyem'
/scratch/s194101/concepts/TUEV/eyem/00000878_00000001_7.pkl
onset=2.9, duration=1.0, description='eyem'
/scratch/s194101/concepts/TUEV/eyem/00000878_00000001_8.pkl
onset=3.0, duration=1.0, description='eyem'
/scratch/s194101/con

KeyboardInterrupt: 

In [ ]:
os.path.basename(sample_file)

'00000081_00000005.edf'